In [1]:
import os
os.chdir('/app/Pogona_realtime/Arena')

In [23]:
import numpy as np
import pandas as pd
import imp
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import time
from tqdm.auto import tqdm
from functools import partial

from Prediction import dataset
from Prediction import train_eval
from Prediction import predictor
from Prediction import visualize
from Prediction import dataset
from Prediction import kalman_predict
from Prediction import seq2seq_predict
from Prediction import calibration as calib
from Prediction import traj_models

In [120]:
imp.reload(dataset)
imp.reload(train_eval)
imp.reload(seq2seq_predict)
imp.reload(predictor)
imp.reload(visualize)
imp.reload(dataset)
imp.reload(kalman_predict)
imp.reload(traj_models)

<module 'Prediction.traj_models' from '/app/Pogona_realtime/Arena/Prediction/traj_models.py'>

# Init Predictor

In [121]:
kalman_predictor = kalman_predict.KalmanPredictor(forecast_horizon=20, 
                                                  num_derivatives=1, 
                                                  q_var=2.145, r_var=120.0)
pred = predictor.HitPredictor(kalman_predictor, dir_path='logs')

In [108]:
trial_paths = dataset.select_analyzed_trials_paths()
trial_names = [dataset.trial_path_to_name(p) for p in trial_paths]
df = dataset.load_trial_data(trial_names)


Loaded 304 trials. 18 were not loaded.


# Test Run Predictor

In [21]:
tdf = df.loc[df.index.unique()[0]]
bbox_cols = [tdf.columns.get_loc(c) for c in ['x1', 'y1', 'x2', 'y2']]

In [122]:
for i in tqdm(range(512)):
    pred.handle_detection(tdf.iloc[i, bbox_cols])
    time.sleep(0.016)

In [123]:
pred.save_predictions()

(512, 20, 4)


In [126]:
fdf = pd.read_csv('logs/forecasts.csv', header=[0,1] )

In [145]:
# access specific multiindex columns
fdf.iloc[:, fdf.columns.get_level_values(0)=='19']

19                                      
              x1           y1          x2           y2
0            NaN          NaN         NaN          NaN
1    1082.549567  2073.400834  857.639888  1871.719505
2    1082.882458  2070.833907  857.167160  1879.214336
3    1083.192982  2061.154178  858.504650  1889.599855
4    1085.600261  2052.201837  856.187315  1902.649009
..           ...          ...         ...          ...
507   708.974228  1269.596624  441.147755  1005.778180
508   710.469591  1274.324800  441.653990   996.408320
509   710.079273  1274.405108  441.056655   991.071961
510   705.982952  1267.134554  449.545110   947.739914
511   701.789456  1263.536273  454.532812   923.477787

[512 rows x 4 columns]